# Note:
-(make sure both data1 and data2 have the same number of numerical and categorical columns)
- Run all the cells for data1
- Then, run all the cells for data2
- Then after you have the correlation matrices for both (data1 and data2) --> run the difference between matrices test
- This is the only way this file will work. If you dont follow the steps you will get a result of ZERO

In [212]:
import numpy as np
import pandas as pd
from numpy import linalg
from sklearn.model_selection import train_test_split

# Upload Dataset

In [215]:
# Dataset before augmentation
data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_orig.csv", header=None)
#data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_orig.csv", header=None)

# Dataset after augmentation
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_new.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_new.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_orig.csv", header=None)
data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_new.csv", header=None)



In [217]:
# Print shapes for debugging
print(f"Shape of data1: {data1.shape}")
print(f"Shape of data2: {data2.shape}")


Shape of data1: (240, 25)
Shape of data2: (172, 13)


In [219]:
# Subset the last 13 columns for the dataset with the 25 columns 
#(we need to make sure we have the same categorical and numerical columns)
data1 = data1.iloc[:, -13:]
data1.shape

(240, 13)

# Convert columns into categorical (Only for Uniform and Gaussian Distribution)

In [163]:
for i in range(-5,0):
    data1.iloc[:,i] = data1.iloc[:,i].astype(int) # Integer
    data1.iloc[:,i] = data1.iloc[:,i].astype('category') # Categories
data1.dtypes
#data2.head()  

0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8      float64
9      float64
10     float64
11     float64
12     float64
13     float64
14     float64
15     float64
16    category
17    category
18    category
19    category
20    category
21    category
22    category
23    category
24    category
dtype: object

In [221]:
for i in range(-5,0):
    data2.iloc[:,i] = data2.iloc[:,i].astype(int) # Integer
    data2.iloc[:,i] = data2.iloc[:,i].astype('category') # Categories
data2.dtypes
#data2.head()  

0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8     category
9     category
10    category
11    category
12    category
dtype: object

# Convert Last 9 columns to Categorical (Only for Stacked Distribution)

In [223]:
#data1
# Convert the last 9 columns to categorical
for column in data1.columns[-5:]:
    data1[column] = data1[column].astype('category')

# Verify the changes
print(data1.dtypes)


12     float64
13     float64
14     float64
15     float64
16     float64
17     float64
18     float64
19     float64
20    category
21    category
22    category
23    category
24    category
dtype: object


# Concat Original dataset with New generated dataset

In [227]:
# Concat (adding rows)
combined_df = np.vstack((data1, data2))

# Reset the index to ensure a clean, continuous index
#combined_df.reset_index(drop=True, inplace=True)
combined_df.shape
combined_df

# change numpy array into a dataframe
combined_data = pd.DataFrame(combined_df, columns=data1.columns)
combined_data

# Convert the last 9 columns to categorical
for column in combined_data.columns[-5:]:
    combined_data[column] = combined_data[column].astype('category')

# Verify the changes
print(combined_data.dtypes)

12     float64
13     float64
14     float64
15     float64
16     float64
17     float64
18     float64
19     float64
20    category
21    category
22    category
23    category
24    category
dtype: object


# Split Dataset

In [229]:
# Split dataset into X and y (data1)
X = combined_data.iloc[:, :-1]  # All columns except the last one
y = combined_data.iloc[:, -1]   # Only the last column


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [203]:
# data2
X = data1.iloc[:, :-1]
y = data1.iloc[:, -1]


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [231]:
# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])

# Correlation between columns test (Matrix)

In [234]:
##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
correlation_matrix = num_corr(numerical_df)

# Print the dataframe to verify
#print(correlation_matrix)
#0.860


---------------------------Correlation Matrix------------------------- 
           12        13        14        15        16        17        18  \
12  1.000000  0.943284  0.939711  0.934689  0.898560  0.870749  0.899566   
13  0.943284  1.000000  0.936787  0.932852  0.898200  0.869270  0.902307   
14  0.939711  0.936787  1.000000  0.922441  0.897029  0.868319  0.888365   
15  0.934689  0.932852  0.922441  1.000000  0.880236  0.865008  0.876035   
16  0.898560  0.898200  0.897029  0.880236  1.000000  0.818155  0.852094   
17  0.870749  0.869270  0.868319  0.865008  0.818155  1.000000  0.829553   
18  0.899566  0.902307  0.888365  0.876035  0.852094  0.829553  1.000000   
19  0.901112  0.897352  0.880754  0.889584  0.849885  0.838820  0.854342   

          19  
12  0.901112  
13  0.897352  
14  0.880754  
15  0.889584  
16  0.849885  
17  0.838820  
18  0.854342  
19  1.000000  


# Save Correlation Matrix into a Dataframe

In [236]:
#combined_data
correlation_matrix = numerical_df.corr()
correlation_df_combined = pd.DataFrame(correlation_matrix)
print(correlation_df_combined)

correlation_df_combined.shape

          12        13        14        15        16        17        18  \
12  1.000000  0.943284  0.939711  0.934689  0.898560  0.870749  0.899566   
13  0.943284  1.000000  0.936787  0.932852  0.898200  0.869270  0.902307   
14  0.939711  0.936787  1.000000  0.922441  0.897029  0.868319  0.888365   
15  0.934689  0.932852  0.922441  1.000000  0.880236  0.865008  0.876035   
16  0.898560  0.898200  0.897029  0.880236  1.000000  0.818155  0.852094   
17  0.870749  0.869270  0.868319  0.865008  0.818155  1.000000  0.829553   
18  0.899566  0.902307  0.888365  0.876035  0.852094  0.829553  1.000000   
19  0.901112  0.897352  0.880754  0.889584  0.849885  0.838820  0.854342   

          19  
12  0.901112  
13  0.897352  
14  0.880754  
15  0.889584  
16  0.849885  
17  0.838820  
18  0.854342  
19  1.000000  


(8, 8)

In [210]:
#data1
correlation_matrix = numerical_df.corr()
correlation_df1 = pd.DataFrame(correlation_matrix)
print(correlation_df1)

correlation_df1.shape

          12        13        14        15        16        17        18  \
12  1.000000  0.236666  0.273645  0.318585  0.123840  0.046459  0.023146   
13  0.236666  1.000000  0.247445  0.269546  0.101889 -0.092261 -0.066529   
14  0.273645  0.247445  1.000000  0.441324  0.015980  0.021592 -0.079043   
15  0.318585  0.269546  0.441324  1.000000  0.040692  0.038481 -0.051545   
16  0.123840  0.101889  0.015980  0.040692  1.000000 -0.041861  0.016662   
17  0.046459 -0.092261  0.021592  0.038481 -0.041861  1.000000  0.073212   
18  0.023146 -0.066529 -0.079043 -0.051545  0.016662  0.073212  1.000000   
19  0.096953 -0.117292 -0.023616  0.066835  0.044408  0.031343  0.034735   

          19  
12  0.096953  
13 -0.117292  
14 -0.023616  
15  0.066835  
16  0.044408  
17  0.031343  
18  0.034735  
19  1.000000  


(8, 8)

In [ ]:
#data2
correlation_matrix = numerical_df.corr()
correlation_df2 = pd.DataFrame(correlation_matrix)
print(correlation_df2)

correlation_df2.shape

# Differences in Matrices (Frobenius Norm)

In [238]:
# Convert the dataframes to numpy arrays
matrix1 = correlation_df1.to_numpy()
matrix2 = correlation_df_combined.to_numpy()

# Compute the Frobenius norm of the difference between the matrices. 
frobenius_norm = np.linalg.norm(matrix1 - matrix2, ord='fro')
#axis= 1 goes across columns
#axis = 0 goes across rows


print(f"Frobenius norm difference: {frobenius_norm: .3f}")


Frobenius norm difference:  6.139


# Differences in Arrays (L2 Norm)

In [ ]:
#array1 ...
#array2 ...

# Compute the L2 norm (Euclidean norm) of the difference between the arrays
L2_norm = np.linalg.norm(array1 - array2, ord=2)

print(f"L2 norm difference: {L2_norm: .3f}")
